# now do some ML using the web scraped data

In [177]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [178]:
from player_data_expansion import get_all_expanded_player_fight_data

In [179]:
# AWS integration getting file data
df_init = pd.read_csv("fight_data.csv").fillna(pd.NA)

In [180]:
df_init.head()

,Date,fighter1,fighter1_result,method_or_round,fighter2,Venue,Undercard fights,link,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,fighter1_debut,fighter2_debut
0,"Sunday, 4 May 2025",Naoya Inoue,Winner,"TKO, Round 8",Ramon Cardenas,"T-Mobile Arena, Las Vegas, United States",Rafael Espinoza Beat Edward Vazquez (TKO 7) \...,https://box.live/fights/inoue-vs-cardenas/,29,0,0,26,0,1,164,168.0,0,0,2012,2015
1,"Saturday, 3 May 2025",Canelo Alvarez,Winner,"UD, Round 12",William Scull,"The Venue, Riyadh, Saudi Arabia",Jaime Munguia Beat Bruno Surace (UD 12) \nMar...,https://box.live/fights/canelo-vs-scull/,62,2,2,23,0,0,175,182.0,0,0,2005,2016
2,"Saturday, 3 May 2025",Jukiya Iimura,Winner,"UD, Round 12",Esneth Domingo,"Korakuen Hall, Tokyo, Japan",<NA>,https://box.live/fights/iimura-vs-domingo/,8,0,1,21,0,2,164,160.0,0,0,2021,2015
3,"Friday, 2 May 2025",Ryan Garcia,Loser,"UD, Round 12",Rolando ‘Rolly’ Romero,"Times Square, New York, NY, United States",Devin Haney Beat Jose Ramirez (UD 12) \nTeofi...,https://box.live/fights/garcia-vs-romero/,24,0,1,16,0,2,178,173.0,83,81,2016,2016
4,"Thursday, 1 May 2025",Kevin Lerena,Winner,"TKO, Round 3",Serhiy Radchenko,"Sunbet Arena, Pretoria, South Africa",<NA>,https://box.live/fights/lerena-vs-radchenko/,30,0,3,11,0,7,185,186.0,47,45,2011,2014


In [181]:
df_init.tail()

,Date,fighter1,fighter1_result,method_or_round,fighter2,Venue,Undercard fights,link,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,fighter1_debut,fighter2_debut
7,"Sunday, 20 April 2025",Ben Whittaker,Winner,"TKO, Round 2",Liam Cameron,"Resorts World Arena, Birmingham, United Kingdom",Sam Eggington Beat Lee Cutler (TD 9) \nFrazer...,https://box.live/fights/whittaker-vs-cameron-ii/,8,1,0,23,1,6,191,183.0,63,43,2022,2009
8,"Saturday, 19 April 2025",Dalton Smith,Winner,"UD, Round 12",Mathieu Germain,"Park Community Arena, Sheffield, United Kingdom",Caoimhin Agyarko Beat Ryan Kelly (MD 10) \nJo...,https://box.live/fights/smith-vs-germain/,17,0,0,26,1,2,176,NaN,0,0,2019,2015
9,"Saturday, 19 April 2025",Gabriela Fundora,Winner,"TKO, Round 7",Marilyn Badillo Amaya,"Frontwave Arena, Oceanside, CA, United States",Jorge Perez Beat Charles Conwell (SD 12) \nOl...,https://box.live/fights/fundora-vs-amaya/,15,0,0,19,1,0,175,160.0,0,0,2021,2017
10,"Saturday, 19 April 2025",Sadriddin Akhmedov,Draw,"MD, Round 10",Elias Espadas,"Commerce Casino, Commerce, CA, United States",<NA>,https://box.live/fights/akhmedov-vs-espadas/,15,0,0,23,0,6,175,178.0,0,0,2018,2012
11,"Friday, 18 April 2025",Sergey Kovalev,Winner,"KO, Round 7",Artur Mann,"Yunost Arena, Chelyabinsk, Russian Federation",<NA>,https://box.live/fights/kovalev-vs-mann/,35,1,5,22,0,4,183,185.0,83,59,2009,2015


In [182]:
# ADD BASIC DATA TO SEE IF IT WORKS
df_init = pd.concat([pd.read_csv("fight_data_backup_basic.csv"), df_init], axis=0)
df_init.drop_duplicates(inplace=True)

### start cleaning

In [183]:
df_essentials = df_init.drop(columns=['link','Venue','Date','Undercard fights','fighter1','fighter2'])
df_essentials.head(2)

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,fighter1_debut,fighter2_debut
0,Winner,"RTD, Round 3",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winner,"UD, Round 12",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
df_staged_cleaning = df_essentials.copy()
df_staged_cleaning.head(2)

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,fighter1_debut,fighter2_debut
0,Winner,"RTD, Round 3",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winner,"UD, Round 12",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
df_staged_cleaning['method_or_round'] = df_staged_cleaning['method_or_round'].str.split(' ')

In [186]:
last_rounds = [new_list[2] for new_list in df_staged_cleaning['method_or_round']][:]
# df_staged_cleaning['round'] = last_rounds round includes test bias

df_staged_cleaning.head()

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,fighter1_debut,fighter2_debut
0,Winner,"[RTD,, Round, 3]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winner,"[UD,, Round, 12]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Winner,"[TKO,, Round, 2]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Winner,"[UD,, Round, 12]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Winner,"[TKO,, Round, 7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
df_staged_cleaning = df_staged_cleaning.drop(columns='method_or_round')
df_staged_cleaning.head()

,fighter1_result,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,fighter1_debut,fighter2_debut
0,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### feature engineering swapped values + add to current values

In [188]:

df_staged_swapping = df_staged_cleaning.copy()

fighter2_wins_temp = df_staged_swapping['fighter2_record_wins'].copy()
fighter2_draws_temp = df_staged_swapping['fighter2_record_draws'].copy()
fighter2_losses_temp = df_staged_swapping['fighter2_record_losses'].copy()
fighter2_height_cm_temp = df_staged_swapping['fighter2_height_cm'].copy()
fighter2_ko_percentage_temp = df_staged_swapping['fighter2_ko_percentage'].copy()
fighter2_debut_temp = df_staged_swapping['fighter2_debut'].copy()

df_staged_swapping['fighter2_record_wins'] = df_staged_swapping['fighter1_record_wins']
df_staged_swapping['fighter2_record_draws'] = df_staged_swapping['fighter1_record_draws']
df_staged_swapping['fighter2_record_losses'] = df_staged_swapping['fighter1_record_losses']
df_staged_swapping['fighter2_height_cm'] = df_staged_swapping['fighter1_height_cm']
df_staged_swapping['fighter2_ko_percentage'] = df_staged_swapping['fighter1_ko_percentage']
df_staged_swapping['fighter2_debut'] = df_staged_swapping['fighter1_debut']

df_staged_swapping['fighter1_record_wins'] = fighter2_wins_temp
df_staged_swapping['fighter1_record_draws'] = fighter2_draws_temp
df_staged_swapping['fighter1_record_losses'] = fighter2_losses_temp
df_staged_swapping['fighter1_height_cm'] = fighter2_height_cm_temp
df_staged_swapping['fighter1_ko_percentage'] = fighter2_ko_percentage_temp
df_staged_swapping['fighter1_debut'] = fighter2_debut_temp



def reverse_results(rowval) : 
    if rowval == 'Winner' :
        return 'Loser'
    elif rowval == 'Loser' :
        return 'Winner'
    else :
        return 'Draw'
    
df_staged_swapping['fighter1_result'] = df_staged_swapping['fighter1_result'].apply(reverse_results)


concatenate the matrices

In [189]:
# df_encoded = pd.concat([df_staged_cleaning, df_staged_swapping], axis=0)
# df_encoded

could also feature engineer the ratio between fighters, such as fighter2/fighter1 for height

### encoding

In [190]:
# commented out. this is the original with no swapping
df_encoded = df_staged_cleaning.copy()

In [191]:
# WINNER = 0, LOSER = 1, DRAW = 2
def custom_encoder(value) :
    if value == 'Winner' :
        return 0
    elif value == 'Loser' :
        return 1
    else : return 2
df_encoded['fighter1_result'] = df_encoded['fighter1_result'].apply(custom_encoder)

In [192]:
df_encoded.shape

(24, 13)

In [193]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 0 to 11
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fighter1_result         24 non-null     int64  
 1   fighter1_record_wins    12 non-null     float64
 2   fighter1_record_draws   12 non-null     float64
 3   fighter1_record_losses  12 non-null     float64
 4   fighter2_record_wins    12 non-null     float64
 5   fighter2_record_draws   12 non-null     float64
 6   fighter2_record_losses  12 non-null     float64
 7   fighter1_height_cm      12 non-null     float64
 8   fighter2_height_cm      11 non-null     float64
 9   fighter1_ko_percentage  12 non-null     float64
 10  fighter2_ko_percentage  12 non-null     float64
 11  fighter1_debut          12 non-null     float64
 12  fighter2_debut          12 non-null     float64
dtypes: float64(12), int64(1)
memory usage: 2.6 KB


In [194]:
df_encoded = df_encoded.infer_objects()

df_encoded['fighter1_result'] = df_encoded['fighter1_result'].astype('category')

df_encoded['fighter1_record_wins'] = df_encoded['fighter1_record_wins'].astype('Int64')
df_encoded['fighter1_record_draws'] = df_encoded['fighter1_record_draws'].astype('Int64')
df_encoded['fighter1_record_losses'] = df_encoded['fighter1_record_losses'].astype('Int64')
df_encoded['fighter2_record_wins'] = df_encoded['fighter2_record_wins'].astype('Int64')
df_encoded['fighter2_record_draws'] = df_encoded['fighter2_record_draws'].astype('Int64')
df_encoded['fighter2_record_losses'] = df_encoded['fighter1_record_losses'].astype('Int64')
df_encoded['fighter1_height_cm'] = df_encoded['fighter2_height_cm'].astype('Int64')
df_encoded['fighter2_height_cm'] = df_encoded['fighter2_height_cm'].astype('Int64')
df_encoded['fighter1_ko_percentage'] = df_encoded['fighter1_ko_percentage'].astype('Int64')
df_encoded['fighter2_ko_percentage'] = df_encoded['fighter2_ko_percentage'].astype('Int64')
# df_encoded['round'] = df_encoded['round'].astype('Int64') round includes test bias
df_encoded['fighter1_debut'] = df_encoded['fighter1_debut'].astype('Int64')
df_encoded['fighter2_debut'] = df_encoded['fighter1_debut'].astype('Int64')

In [195]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 0 to 11
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   fighter1_result         24 non-null     category
 1   fighter1_record_wins    12 non-null     Int64   
 2   fighter1_record_draws   12 non-null     Int64   
 3   fighter1_record_losses  12 non-null     Int64   
 4   fighter2_record_wins    12 non-null     Int64   
 5   fighter2_record_draws   12 non-null     Int64   
 6   fighter2_record_losses  12 non-null     Int64   
 7   fighter1_height_cm      11 non-null     Int64   
 8   fighter2_height_cm      11 non-null     Int64   
 9   fighter1_ko_percentage  12 non-null     Int64   
 10  fighter2_ko_percentage  12 non-null     Int64   
 11  fighter1_debut          12 non-null     Int64   
 12  fighter2_debut          12 non-null     Int64   
dtypes: Int64(12), category(1)
memory usage: 2.9 KB


In [196]:
df_encoded.describe()

,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,fighter1_debut,fighter2_debut
count,12.0,12.0,12.0,12.0,12.0,12.0,11.0,11.0,12.0,12.0,12.0,12.0
mean,26.333333,0.333333,1.583333,20.666667,0.333333,1.583333,174.0,174.0,29.166667,24.083333,2014.416667,2014.416667
std,15.399725,0.651339,1.78164,4.579268,0.492366,1.78164,9.591663,9.591663,37.201743,31.154916,5.79119,5.79119
min,8.0,0.0,0.0,11.0,0.0,0.0,160.0,160.0,0.0,0.0,2005.0,2005.0
25%,15.0,0.0,0.0,18.25,0.0,0.0,167.0,167.0,0.0,0.0,2010.5,2010.5
50%,26.5,0.0,1.0,22.5,0.0,1.0,173.0,173.0,0.0,0.0,2014.0,2014.0
75%,34.25,0.25,3.0,23.0,1.0,3.0,182.5,182.5,65.75,48.5,2019.5,2019.5
max,62.0,2.0,5.0,26.0,1.0,5.0,186.0,186.0,83.0,81.0,2022.0,2022.0


### tt split

In [197]:
from sklearn.model_selection import train_test_split

In [198]:
X = df_encoded.drop(columns=['fighter1_result'])
y = df_encoded['fighter1_result']
y.unique()

[0, 2, 1]
Categories (3, int64): [0, 1, 2]

In [199]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66) # tweak to maker sure that y values contain wins, losses, and draws. ts = .2 and rs =  66
print(y_train.unique())
print(y_test.unique())

[0, 2, 1]
Categories (3, int64): [0, 1, 2]
[0, 2]
Categories (3, int64): [0, 1, 2]


## TRAIN MODEL - XGBoost First

In [200]:
from xgboost import XGBClassifier

In [201]:
xgbc = XGBClassifier(
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1
)

In [202]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [203]:
y_train = encoder.fit_transform(y_train)

In [204]:
# compute sample weight based on class frequency
class_weights = dict(enumerate(len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train))))
sample_weight = np.array([class_weights[label] for label in y_train])
sample_weight

array([0.37254902, 0.37254902, 0.37254902, 0.37254902, 0.37254902,
       0.37254902, 0.37254902, 0.37254902, 0.37254902, 0.37254902,
       0.37254902, 0.37254902, 0.37254902, 0.37254902, 6.33333333,
       6.33333333, 0.37254902, 0.37254902, 0.37254902])

In [205]:
# # no class weighting
# xgbc.fit(X_train, y_train)

# class weighting
xgbc.fit(X_train, y_train, sample_weight=sample_weight)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=3, ...)

In [206]:
xgbc_preds = xgbc.predict(X_test)
xgbc_preds = encoder.inverse_transform(xgbc_preds)

In [207]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [208]:
print(accuracy_score(y_test, xgbc_preds))

0.8


In [209]:
print(confusion_matrix(y_test, xgbc_preds))

[[4 0]
 [1 0]]


In [210]:
print(classification_report(y_test, xgbc_preds))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           2       0.00      0.00      0.00         1

    accuracy                           0.80         5
   macro avg       0.40      0.50      0.44         5
weighted avg       0.64      0.80      0.71         5



c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [211]:
xgbc_preds

array([0, 0, 0, 0, 0], dtype=int64)

### grid search cross validated + XGBoost Classifier - same rseult as initial one

In [212]:
from sklearn.model_selection import GridSearchCV

In [213]:
eval_set = [(X, y)] # validation set for early stopping

In [214]:
xgbc_model2 = XGBClassifier(
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1,
    eval_metric="mlogloss"
)

In [215]:
param_grid = {
    "subsample":[0.75, 1],
     "colsample_bytree":[0.75, 1],
     "max_depth":[2, 6],
     "min_child_weight":[1, 2,3],
     "learning_rate":[1, 0.1, 0.01],
}

In [216]:
xgbc_grid = GridSearchCV(
    xgbc_model2,
    param_grid,
    cv=3,
    scoring="f1"
)

In [217]:
xgbc_grid.fit(X_train,y_train,eval_set=eval_set,verbose=1)

[0]	validation_0-mlogloss:0.55051
[1]	validation_0-mlogloss:0.46267
[2]	validation_0-mlogloss:0.44749
[3]	validation_0-mlogloss:0.44064
[4]	validation_0-mlogloss:0.40557
[5]	validation_0-mlogloss:0.41480
[6]	validation_0-mlogloss:0.38421
[7]	validation_0-mlogloss:0.38417
[8]	validation_0-mlogloss:0.38801
[9]	validation_0-mlogloss:0.38726
[10]	validation_0-mlogloss:0.38483


[11]	validation_0-mlogloss:0.38177
[12]	validation_0-mlogloss:0.39516
[13]	validation_0-mlogloss:0.39663
[14]	validation_0-mlogloss:0.39526
[15]	validation_0-mlogloss:0.40927
[16]	validation_0-mlogloss:0.39938
[17]	validation_0-mlogloss:0.39647
[18]	validation_0-mlogloss:0.38830
[19]	validation_0-mlogloss:0.38452
[20]	validation_0-mlogloss:0.39106
[21]	validation_0-mlogloss:0.38562
[22]	validation_0-mlogloss:0.38809
[23]	validation_0-mlogloss:0.38804
[24]	validation_0-mlogloss:0.38465
[25]	validation_0-mlogloss:0.38809
[26]	validation_0-mlogloss:0.40560
[27]	validation_0-mlogloss:0.40884
[28]	validation_0-mlogloss:0.40673
[29]	validation_0-mlogloss:0.41301
[30]	validation_0-mlogloss:0.40793
[31]	validation_0-mlogloss:0.41742


c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


[32]	validation_0-mlogloss:0.42297
[33]	validation_0-mlogloss:0.41181
[34]	validation_0-mlogloss:0.41397
[35]	validation_0-mlogloss:0.41704
[36]	validation_0-mlogloss:0.43052
[37]	validation_0-mlogloss:0.41754
[38]	validation_0-mlogloss:0.41366
[39]	validation_0-mlogloss:0.42211
[40]	validation_0-mlogloss:0.42420
[41]	validation_0-mlogloss:0.42405
[42]	validation_0-mlogloss:0.42711
[43]	validation_0-mlogloss:0.42668
[44]	validation_0-mlogloss:0.42211
[45]	validation_0-mlogloss:0.42418
[46]	validation_0-mlogloss:0.43129
[47]	validation_0-mlogloss:0.42537
[48]	validation_0-mlogloss:0.44052
[49]	validation_0-mlogloss:0.44478
[50]	validation_0-mlogloss:0.44247
[51]	validation_0-mlogloss:0.44064
[52]	validation_0-mlogloss:0.43570
[53]	validation_0-mlogloss:0.43177
[54]	validation_0-mlogloss:0.42618
[55]	validation_0-mlogloss:0.42284
[56]	validation_0-mlogloss:0.42949
[57]	validation_0-mlogloss:0.42977
[58]	validation_0-mlogloss:0.43790
[59]	validation_0-mlogloss:0.43625
[60]	validation_0-ml

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

...

[1]	validation_0-mlogloss:1.08155
[2]	validation_0-mlogloss:1.07326
[3]	validation_0-mlogloss:1.06515
[4]	validation_0-mlogloss:1.05700
[5]	validation_0-mlogloss:1.04892
[6]	validation_0-mlogloss:1.04132
[7]	validation_0-mlogloss:1.03377
[8]	validation_0-mlogloss:1.02543
[9]	validation_0-mlogloss:1.01740
[10]	validation_0-mlogloss:1.00879
[11]	validation_0-mlogloss:1.00157
[12]	validation_0-mlogloss:0.99451
[13]	validation_0-mlogloss:0.98753
[14]	validation_0-mlogloss:0.98073
[15]	validation_0-mlogloss:0.97418
[16]	validation_0-mlogloss:0.96747
[17]	validation_0-mlogloss:0.96236
[18]	validation_0-mlogloss:0.95594
[19]	validation_0-mlogloss:0.94961
[20]	validation_0-mlogloss:0.94272
[21]	validation_0-mlogloss:0.93646
[22]	validation_0-mlogloss:0.92947
[23]	validation_0-mlogloss:0.92679
[24]	validation_0-mlogloss:0.92080
[25]	validation_0-mlogloss:0.91492
[26]	validation_0-mlogloss:0.90896
[27]	validation_0-mlogloss:0.90318
[28]	validation_0-mlogloss:0.89682
[29]	validation_0-mlogloss:0.

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[1]	validation_0-mlogloss:1.08050
[2]	validation_0-mlogloss:1.07166
[3]	validation_0-mlogloss:1.06295
[4]	validation_0-mlogloss:1.05438
[5]	validation_0-mlogloss:1.04595
[6]	validation_0-mlogloss:1.03764
[7]	validation_0-mlogloss:1.02946
[8]	validation_0-mlogloss:1.02141
[9]	validation_0-mlogloss:1.01348
[10]	validation_0-mlogloss:1.00567
[11]	validation_0-mlogloss:0.99798
[12]	validation_0-mlogloss:0.99040
[13]	validation_0-mlogloss:0.98294
[14]	validation_0-mlogloss:0.97559
[15]	validation_0-mlogloss:0.96835
[16]	validation_0-mlogloss:0.96122
[17]	validation_0-mlogloss:0.95419
[18]	validation_0-mlogloss:0.94727
[19]	validation_0-mlogloss:0.94045
[20]	validation_0-mlogloss:0.93373
[21]	validation_0-mlogloss:0.92711
[22]	validation_0-mlogloss:0.92059
[23]	validation_0-mlogloss:0.91416
[24]	validation_0-mlogloss:0.90782
[25]	validation_0-mlogloss:0.90158
[26]	validation_0-mlogloss:0.89543
[27]	validation_0-mlogloss:0.88937
[28]	validation_0-mlogloss:0.88339
[29]	validation_0-mlogloss:0.

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constrain...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_class=3, ...),
             param_grid={'colsample_bytree': [0.75, 1],
                         'learning_rate': [1, 0.1, 0.01], 'max_depth': [2, 6],
                         'min_child_weight': [1, 2, 3],
                         'subsample': [0.75, 1]},
             scoring='f1')

In [218]:
xgbc_grid.best_params_

{'colsample_bytree': 0.75,
 'learning_rate': 1,
 'max_depth': 2,
 'min_child_weight': 1,
 'subsample': 0.75}

In [219]:
xgbc_cv_best = XGBClassifier(
    colsample_bytree = 0.75,
    learning_rate = 1,
    max_depth = 2,
    min_child_weight = 3,
    subsample = 0.75,
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1,
    eval_metric="mlogloss"
)

In [220]:
xgbc_cv_best.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=3, ...)

In [221]:
xgbc_cv_best_preds = xgbc_cv_best.predict(X_test)
xgbc_cv_best_preds = encoder.inverse_transform(xgbc_cv_best_preds)

In [222]:
print(confusion_matrix(y_test, xgbc_cv_best_preds),"\n")
print(classification_report(y_test, xgbc_cv_best_preds))

[[4 0]
 [1 0]] 

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           2       0.00      0.00      0.00         1

    accuracy                           0.80         5
   macro avg       0.40      0.50      0.44         5
weighted avg       0.64      0.80      0.71         5



c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### random forest

In [223]:
from sklearn.ensemble import RandomForestClassifier

In [224]:
X_train.shape[1]

12

In [225]:
rfc_1 = RandomForestClassifier(
    n_estimators=round(np.sqrt(X_train.shape[1])),
    random_state=42
    )

In [226]:
rfc_1.fit(X_train,y_train)

RandomForestClassifier(n_estimators=3, random_state=42)

In [227]:
rfc_1_preds = rfc_1.predict(X_test)

In [228]:
rfc_1_preds = encoder.inverse_transform(rfc_1_preds)

In [229]:
print(classification_report(y_test,rfc_1_preds))
print(confusion_matrix(y_test,rfc_1_preds))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           2       0.00      0.00      0.00         1

    accuracy                           0.80         5
   macro avg       0.40      0.50      0.44         5
weighted avg       0.64      0.80      0.71         5

[[4 0]
 [1 0]]


c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
